In [0]:
import os
import numpy as np
import pandas as pd
import cv2
from zipfile import ZipFile 

# file_name = '//9d34462453e311ea.zip'

# with ZipFile(file_name, 'r') as zip:
#   zip.extractall()  

In [34]:
 path = '/content/dataset'

 train_df = pd.read_csv(path+'/train.csv')
 test_df = pd.read_csv(path+'/test.csv')

 train_df.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [35]:
class_map = {
    'Food': 0,
    'Attire': 1,
    'Decorationandsignage': 2,
    'misc': 3
}

inverse_class_map = {
    0: 'Food',
    1: 'Attire',
    2: 'Decorationandsignage',
    3: 'misc'
}

train_df['Class'] = train_df['Class'].map(class_map).astype(np.uint8)
train_df.head()

,Image,Class
0,image7042.jpg,0
1,image3327.jpg,3
2,image10335.jpg,1
3,image8019.jpg,0
4,image2128.jpg,1


In [0]:
train_path = '/content/dataset/Train Images/'
test_path = '/content/dataset/Test Images/'

train_images, train_labels = [], []

for i in range(len(train_df.Image)):
  train_image = cv2.imread(train_path + str(train_df.Image[i]))
  train_image = cv2.resize(train_image, (224, 224))
  train_images.append(train_image)
  train_labels.append(train_df.Class[i])

test_images = []

for i in range(len(test_df.Image)):
  test_image = cv2.imread(test_path + str(test_df.Image[i]))
  test_image = cv2.resize(test_image, (224, 224))
  test_images.append(test_image)

train_images = np.array(train_images)
test_images = np.array(test_images)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_images, to_categorical(train_labels), test_size=0.2, random_state=42)

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [0]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False, 
    input_shape=(224, 224, 3), 
    pooling='avg'
)
# base_model.summary()

In [40]:
base_model.trainable = False

model = Sequential([
  base_model,
  Dense(4, activation='softmax'),
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 8196      
Total params: 23,595,908
Trainable params: 8,196
Non-trainable params: 23,587,712
_________________________________________________________________


In [42]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), callbacks=[EarlyStopping(min_delta=0.01)])

Train on 4786 samples, validate on 1197 samples
Epoch 1/5
4786/4786 [==============================] - 1509s 315ms/step - loss: 0.6887 - acc: 0.7457 - val_loss: 0.7546 - val_acc: 0.7210
Epoch 2/5
4786/4786 [==============================] - 1515s 317ms/step - loss: 0.5599 - acc: 0.7963 - val_loss: 0.7319 - val_acc: 0.7444
Epoch 3/5
4786/4786 [==============================] - 1505s 314ms/step - loss: 0.4997 - acc: 0.8180 - val_loss: 0.7711 - val_acc: 0.7285


In [43]:
labels = model.predict(test_images)
print(labels[:4])

[[1.6480432e-01 6.2477797e-01 4.5917789e-04 2.0995848e-01]
 [7.8995579e-01 1.7900714e-01 9.7828612e-05 3.0939169e-02]
 [8.3705401e-01 1.0040668e-02 1.2289748e-03 1.5167642e-01]
 [9.9659699e-01 2.7689716e-04 3.8049639e-05 3.0880684e-03]]


In [44]:
label = [np.argmax(i) for i in labels]
print(label[:20])

[1, 0, 0, 0, 1, 3, 3, 1, 3, 1, 2, 2, 2, 1, 1, 1, 1, 3, 3, 1]


In [45]:
class_label = [inverse_class_map[x] for x in label]
print(class_label[:3])

['Attire', 'Food', 'Food']


In [46]:
submission = pd.DataFrame({ 'Image': test_df.Image, 'Class': class_label })
submission.head()

,Image,Class
0,image6245.jpg,Attire
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire


In [0]:
submission.to_csv('sub.csv', index=False)